<a href="https://colab.research.google.com/github/yousrabougara/WalmartSalesForecasting/blob/LightGBM/LightGBM_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import StandardScaler ,PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression , Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from xgboost import XGBRegressor
from category_encoders import BinaryEncoder
import pickle
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('Walmart.csv')

In [ ]:
df.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [ ]:
df.shape

(6435, 8)

In [ ]:
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [ ]:
df.fillna(0, inplace=True)

In [ ]:
def get_season(month):
    '''this function is to get season according to month column'''
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Season'] = df['Month'].apply(get_season)

In [ ]:
df['Store'] = df['Store'].astype('object')

In [ ]:
pip install lightgbm

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error


In [ ]:
num_vars = ['Temperature','Fuel_Price','CPI','Unemployment','Weekly_Sales']

# scaling numerical variables
sc = StandardScaler()
df[num_vars] = sc.fit_transform(df[num_vars])

# encoding categorical features
encoder = BinaryEncoder(cols=['Store','Season'])
df = encoder.fit_transform(df)

In [ ]:
features = df.columns.drop(['Weekly_Sales','Date','Year'])
target = 'Weekly_Sales'

X = df[features]
y = df[target]

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42 ,  test_size=0.2)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score


# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the LightGBM model using the early_stopping callback
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)],  # Pass verbose_eval to log_evaluation callback
)

# Predict on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Evaluate the model
# Evaluate the model
if params['objective'] == 'regression':
    # Calculate RMSE without the 'squared' argument for older scikit-learn versions
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"RMSE: {rmse}")
else:  # For classification
    y_pred_binary = (y_pred > 0.5).astype(int) if params['objective'] == 'binary' else y_pred.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred_binary)
    print(f"Accuracy: {accuracy}")

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.194004	valid_1's rmse: 0.222503
[200]	training's rmse: 0.15189	valid_1's rmse: 0.200784
[300]	training's rmse: 0.132816	valid_1's rmse: 0.195573
[400]	training's rmse: 0.120153	valid_1's rmse: 0.192256
[500]	training's rmse: 0.110815	valid_1's rmse: 0.190107
[600]	training's rmse: 0.103054	valid_1's rmse: 0.189216
Early stopping, best iteration is:
[577]	training's rmse: 0.104603	valid_1's rmse: 0.1891
RMSE: 0.18910035949236356


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the LightGBM model using the early_stopping callback
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
)

# Predict on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Evaluate the model
# Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
# R2 Score
r2 = r2_score(y_test, y_pred)

# Print results
print("Final Test R2 Score:", r2)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)

# Training and test set accuracy
train_score = model.predict(X_train, num_iteration=model.best_iteration)
train_r2 = r2_score(y_train, train_score)
print("Training set R2 Score:", train_r2)

# Test set accuracy (percentage)
test_r2 = r2
print("Test set R2 Score:", test_r2)


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.194004	valid_1's rmse: 0.222503
[200]	training's rmse: 0.15189	valid_1's rmse: 0.200784
[300]	training's rmse: 0.132816	valid_1's rmse: 0.195573
[400]	training's rmse: 0.120153	valid_1's rmse: 0.192256
[500]	training's rmse: 0.110815	valid_1's rmse: 0.190107
[600]	training's rmse: 0.103054	valid_1's rmse: 0.189216
Early stopping, best iteration is:
[577]	training's rmse: 0.104603	valid_1's rmse: 0.1891
Final Test R2 Score: 0.96465112747043
Mean Squared Error (MSE): 0.035758945960141134
Root Mean Squared Error (RMSE): 0.18910035949236356
Mean Absolute Error (MAE): 0.1110480002021541
Training set R2 Score: 0.9890256661912437
Test set R2 Score: 0.96465112747043


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
        'verbose': -1
    }

    # Create LightGBM datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)

    # Train the model using callbacks for early stopping
    model = lgb.train(
        params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, test_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),  # Use callback for early stopping
            lgb.log_evaluation(100)  # Log evaluation every 100 iterations
        ]
    )

    # Predict on the test set
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Calculate and return the R² score (Optuna maximizes this score)
    return r2_score(y_test, y_pred)

In [ ]:

# Create an Optuna study to maximize R²
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Display the best hyperparameters found by Optuna
best_params = study.best_params
print("Best parameters found by Optuna:", best_params)

# Train the final model with the best parameters
final_model = lgb.train(
    {**best_params, 'objective': 'regression', 'metric': 'rmse', 'verbose': -1},
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=1000,
    valid_sets=[lgb.Dataset(X_test, label=y_test)],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]
)

# Predict on the test set
y_pred = final_model.predict(X_test, num_iteration=final_model.best_iteration)

# Evaluate the final model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print("Final Test R2 Score:", r2)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)



[I 2025-01-31 11:00:29,650] A new study created in memory with name: no-name-b17cd058-37e8-4128-be5c-fb8bdc508e22


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.284831	valid_1's rmse: 0.322745
[200]	training's rmse: 0.244819	valid_1's rmse: 0.291347
[300]	training's rmse: 0.231526	valid_1's rmse: 0.282253
[400]	training's rmse: 0.226599	valid_1's rmse: 0.279613
Early stopping, best iteration is:
[391]	training's rmse: 0.226599	valid_1's rmse: 0.279613


[I 2025-01-31 11:00:30,189] Trial 0 finished with value: 0.9227130269371832 and parameters: {'learning_rate': 0.09900883160727124, 'num_leaves': 70, 'max_depth': 12, 'feature_fraction': 0.7606435370711273, 'bagging_fraction': 0.9493052728892531, 'bagging_freq': 8, 'min_data_in_leaf': 76, 'lambda_l1': 5.0782302585979835, 'lambda_l2': 6.018551760270656}. Best is trial 0 with value: 0.9227130269371832.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.835341	valid_1's rmse: 0.829521
[200]	training's rmse: 0.781473	valid_1's rmse: 0.774687
[300]	training's rmse: 0.750304	valid_1's rmse: 0.742625
[400]	training's rmse: 0.727155	valid_1's rmse: 0.72074
[500]	training's rmse: 0.701767	valid_1's rmse: 0.696818
[600]	training's rmse: 0.681823	valid_1's rmse: 0.678931
[700]	training's rmse: 0.665523	valid_1's rmse: 0.665084
[800]	training's rmse: 0.651743	valid_1's rmse: 0.652615
[900]	training's rmse: 0.640012	valid_1's rmse: 0.643101


[I 2025-01-31 11:00:32,278] Trial 1 finished with value: 0.6021370431991109 and parameters: {'learning_rate': 0.02558674752400711, 'num_leaves': 44, 'max_depth': 2, 'feature_fraction': 0.8731607809680817, 'bagging_fraction': 0.8909383387708951, 'bagging_freq': 8, 'min_data_in_leaf': 89, 'lambda_l1': 6.587101607007712, 'lambda_l2': 3.3047704805108005}. Best is trial 0 with value: 0.9227130269371832.


...

[900]	training's rmse: 0.673216	valid_1's rmse: 0.670334
[1000]	training's rmse: 0.659896	valid_1's rmse: 0.658101
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.659896	valid_1's rmse: 0.658101
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.52382	valid_1's rmse: 0.533544
[200]	training's rmse: 0.387784	valid_1's rmse: 0.405064
[300]	training's rmse: 0.319751	valid_1's rmse: 0.34453
[400]	training's rmse: 0.286591	valid_1's rmse: 0.315501
[500]	training's rmse: 0.267747	valid_1's rmse: 0.299936
[600]	training's rmse: 0.256778	valid_1's rmse: 0.290944
[700]	training's rmse: 0.249989	valid_1's rmse: 0.285824
[800]	training's rmse: 0.246267	valid_1's rmse: 0.282911
[900]	training's rmse: 0.243866	valid_1's rmse: 0.281325
[1000]	training's rmse: 0.242554	valid_1's rmse: 0.280365
Did not meet early stopping. Best iteration is:
[994]	training's rmse: 0.242608	valid_1's rmse: 0.280347


[I 2025-01-31 11:05:26,805] Trial 98 finished with value: 0.9223055377541315 and parameters: {'learning_rate': 0.01406913143595875, 'num_leaves': 66, 'max_depth': 14, 'feature_fraction': 0.9227758798623903, 'bagging_fraction': 0.8874211338299635, 'bagging_freq': 4, 'min_data_in_leaf': 10, 'lambda_l1': 9.179697060568131, 'lambda_l2': 6.493736726365084}. Best is trial 74 with value: 0.9654621980263614.


Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 0.299876	valid_1's rmse: 0.316407
[200]	training's rmse: 0.18978	valid_1's rmse: 0.229432
[300]	training's rmse: 0.155151	valid_1's rmse: 0.212184
[400]	training's rmse: 0.136103	valid_1's rmse: 0.205915
[500]	training's rmse: 0.12313	valid_1's rmse: 0.200242
[600]	training's rmse: 0.113515	valid_1's rmse: 0.197175
[700]	training's rmse: 0.105963	valid_1's rmse: 0.195177
[800]	training's rmse: 0.0998763	valid_1's rmse: 0.193171
[900]	training's rmse: 0.094853	valid_1's rmse: 0.192321
[1000]	training's rmse: 0.0905261	valid_1's rmse: 0.191467
Did not meet early stopping. Best iteration is:
[984]	training's rmse: 0.0910681	valid_1's rmse: 0.191383


[I 2025-01-31 11:05:31,436] Trial 99 finished with value: 0.9637926857490675 and parameters: {'learning_rate': 0.019555646448923557, 'num_leaves': 83, 'max_depth': 17, 'feature_fraction': 0.9483887818650532, 'bagging_fraction': 0.9561174554154281, 'bagging_freq': 6, 'min_data_in_leaf': 13, 'lambda_l1': 0.2415900743114014, 'lambda_l2': 8.466280115110148}. Best is trial 74 with value: 0.9654621980263614.


Best parameters found by Optuna: {'learning_rate': 0.017642721192508683, 'num_leaves': 74, 'max_depth': 15, 'feature_fraction': 0.9829255444351148, 'bagging_fraction': 0.9277083460449651, 'bagging_freq': 5, 'min_data_in_leaf': 14, 'lambda_l1': 0.01254863852155864, 'lambda_l2': 8.150528543544082}
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.32632
[200]	valid_0's rmse: 0.22945
[300]	valid_0's rmse: 0.20951
[400]	valid_0's rmse: 0.202125
[500]	valid_0's rmse: 0.196558
[600]	valid_0's rmse: 0.193162
[700]	valid_0's rmse: 0.190351
[800]	valid_0's rmse: 0.188874
[900]	valid_0's rmse: 0.188045
[1000]	valid_0's rmse: 0.18693
Did not meet early stopping. Best iteration is:
[995]	valid_0's rmse: 0.186918
Final Test R2 Score: 0.9654621980263614
Mean Squared Error (MSE): 0.03493846637751377
Root Mean Squared Error (RMSE): 0.18691834146897884
Mean Absolute Error (MAE): 0.10724851939126356
